Уходим от списков к массивам

In [1]:
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import random 
#!pip install import-ipynb
#import import_ipynb
import modelparameters as mpar
import numpy as np
#from sympy import *
%load_ext cython
import time
import array
import numpy as np

T 108.33078115826872


In [2]:
%load_ext pythran.magic

In [24]:
%%pythran
import numpy as np
#pythran export foo(float64[],float)
def foo(a,t): 
    rp=a.reshape((2,3,3))
    r=rp[0,:,:]
    p=rp[1,:,:]
    reserved=np.zeros((2,3,3),dtype=np.float64)
    reserved[0,:,:]=p
    reserved[1,:,:]=r
    return reserved

In [3]:
tst=np.array(['x', 'y', 'z', 'x_1', 'y_1', 'z_1', 'x_2', 'y_2', 'z_2', 
              'px', 'py', 'pz', 'p_1x', 'p_1y', 'p_1z', 'p_2x', 'p_2y', 'p_2z'])
print(tst[0:3],'\n')
print(tst[3:6],'\n')
tst=tst.reshape((2,3,3))
print("\n",tst,"\n")
plus=[2,0,1]
minus=[1,2,0]
tstPlus=tst[:,plus,:]
tstMinus=tst[:,minus,:]
print(tst[0,:,:],'\n',tstPlus[0,:,:],'\n',tstMinus[0,:,:])


['x' 'y' 'z'] 

['x_1' 'y_1' 'z_1'] 


 [[['x' 'y' 'z']
  ['x_1' 'y_1' 'z_1']
  ['x_2' 'y_2' 'z_2']]

 [['px' 'py' 'pz']
  ['p_1x' 'p_1y' 'p_1z']
  ['p_2x' 'p_2y' 'p_2z']]] 

[['x' 'y' 'z']
 ['x_1' 'y_1' 'z_1']
 ['x_2' 'y_2' 'z_2']] 
 [['x_2' 'y_2' 'z_2']
 ['x' 'y' 'z']
 ['x_1' 'y_1' 'z_1']] 
 [['x_1' 'y_1' 'z_1']
 ['x_2' 'y_2' 'z_2']
 ['x' 'y' 'z']]


In [5]:

#rhs=np.zeros((2,3,3))
#global rhs

In [7]:
#%%pythran
import numpy as np
m_p=mpar.m_p
aep=mpar.aep
app=mpar.app
M=np.array([[1.0],[m_p],[m_p]])

PPlus  = np.array([2,0,1])
PMinus = np.array([1,2,0])
a2=np.array([[aep**2],[aep**2],[app**2]])
fc1=np.array([1,-1,1]).reshape(3,1)
fc2=np.array([-1,-1,1]).reshape(3,1)
rhs=np.zeros((2,3,3),dtype=np.float64)
global rhs
def  dif(a , t) :
    global rhs
    rp=a.reshape((2,3,3))
    r=rp[0,:,:]
    p=rp[1,:,:]
    dr=r-r[PPlus,:]
    fplus_norm=np.sqrt(np.linalg.norm(dr,axis=1,keepdims=True)**2+a2)**3
    fplus=dr/fplus_norm
    
    ftotal=fplus*fc2 + fplus[PMinus,:]*fc1
    rhs[0,:,:]=p/M
    rhs[1:,:]=ftotal
    return rhs


In [11]:

def  dif_v2(a , t) :
    #x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    rp=a.reshape((2,3,3))
    #print(rp)
    r=rp[0,:,:]
    p=rp[1,:,:]
    dr=r-r[PPlus,:]
    #print("dr=",dr)
    fplusn=np.sqrt(np.linalg.norm(rp[0,:,:]-rp[0,PPlus,:],axis=1,keepdims=True)**2+a2)**3
    #print("fplusn=",fplusn)
    fplus=(rp[0,:,:]-rp[0,PPlus,:])/fplusn
    #print(fplus)
    #print()
    ftotal=fplus[PMinus,:]*fc1+fplus*fc2
    #print(ftotal)
    rhs=np.zeros((2,3,3),dtype=np.float64)
    rhs[0,:,:]=p/M
    rhs[1:,:]=ftotal
    return rhs.flatten()

In [8]:
a_0 =  np.array([0.0, 0.0, 0.0, 
                 0.0, 0.0, 1.0, 
                 0.0, 0.0, -1.0, 
                 0.0, 0.0, 0.0, 0.1, 0.2, 0.3, -0.1, -0.2, -0.3])
t=0.0
%time (dif(a_0,t))

CPU times: user 211 µs, sys: 18 µs, total: 229 µs
Wall time: 183 µs


array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 5.44602984e-05,  1.08920597e-04,  1.63380895e-04],
        [-5.44602984e-05, -1.08920597e-04, -1.63380895e-04]],

       [[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-0.00000000e+00, -0.00000000e+00, -8.69253871e-02],
        [ 0.00000000e+00,  0.00000000e+00,  8.69253871e-02]]])

In [12]:
%%timeit 
d=dif(a_0,t)

50.4 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [ ]:
%%cython -a
cimport numpy as np
from numpy cimport int32_t
from cython.view cimport array as carray
cdef extern from "math.h":
    double sin(double)
    const double pi "M_PI"
cdef float aep = 1.1560
cdef float app = 0.8669
cdef float m_p = 1836.2
cdef float E_0 = 0.6
cdef float w_0 = 0.058
cdef int N = 3
cdef float T = 2 * pi * N /w_0

cdef  double length2(double[:] R,  double[:] r):
    return pow((R[0] - r[0]),2)  + pow((R[1] - r[1]),2) + pow((R[2] - r[2]),2)

cdef  float f_e(int i,  double[:] R_1,  double[:] R_2,  double[:] r):
    return (R_1[i] - r[i]) / (
                pow((length2(R_1,r) + pow(aep,2)), (3 / 2))) + (
                (R_2[i] - r[i])) /  (
                pow((length2(R_2,r) + pow(aep, 2)),(3 / 2)))

cdef float f_p1(int i,  double[:] R_1,  double[:] R_2,  double[:] r) :
        return (-(R_1[i] - r[i]) / (
                pow((length2(R_1,r) + pow(aep,2)),  (3 / 2))) + (R_1[i] - R_2[i]) / pow((length2(R_1,R_2) + pow(app,2)),(3 / 2)))

cdef float f_p2(int i,  double[:] R_1,  double[:] R_2,  double[:] r) :
        return (-(R_2[i] - r[i]) / (
                pow((length2(R_2,r) + pow(aep,2)) , (3 / 2))) - (R_1[i] - R_2[i]) / pow((length2(R_1,R_2) + pow(app,2)),(3 / 2)))



    
    return dadt
cdef float E(t):
    E = E_0 * pow((sin(pi * t / T)),2) * sin(w_0 * t)
    return E

cpdef  dif2(double[:]a, double t2):
    x, y, z, x_1, y_1, z_1, x_2, y_2, z_2, px, py, pz, p_1x, p_1y, p_1z, p_2x, p_2y, p_2z = a
    cdef int L = 100000
    cdef double [3] electron = [x, y, z]
    cdef double [3] proton1 = [x_1, y_1, z_1]
    cdef double [3] proton2 = [x_2, y_2, z_2]
    dadt  = [px, py, pz,
            p_1x / m_p, p_1y / m_p, p_1z / m_p,
            p_2x / m_p, p_2y / m_p, p_2z / m_p,
            f_e(0, proton1,proton2, electron),
            f_e(1, proton1,proton2, electron),
            f_e(2, proton1,proton2, electron)- E(t2),
            f_p1(0,  proton1,proton2, electron),
            f_p1(1,  proton1,proton2, electron),
            f_p1(2,  proton1,proton2, electron) + E(t2), 
            f_p2(0,  proton1,proton2, electron),
            f_p2(1,  proton1,proton2, electron),
            f_p2(2,  proton1,proton2, electron) + E(t2)] 
    return dadt

In [ ]:
p = mpar.p
t = np.linspace(0, 50, 100)
#def a_0(i):
#####a_0 =  np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
a_0 = np.zeros((18),dtype=np.float64)
#print(dif2,'uyi',a_0,'niu', t) 
(odeint(dif, a_0, t))

In [ ]:
p = mpar.p
R_1 = np.array([-mpar.dist/2, 0.0, 0.0]) #начальные положения pr1,pr2,e
R_2 = np.array([mpar.dist/2, 0.0, 0.0])
r = np.array([0,0,0])
i=0

print (mpar.dist)
def a_0(i):
    a_0 = np.array([0.0, 0.0, 0.0,
                    -mpar.dist / 2, 0.0, 0.0,
                    mpar.dist / 2, 0.0, 0.0,
                    p[i][0], p[i][1], p[i][2], 
                    0.0, 0.0, 0.0, 
                    0.0, 0.0, 0.0])
    return a_0
solution =  []
start = time.time()
odeint(dif, a_0(i), mpar.t)
#Решения 18 уравнений для n экспериментов
for i in range (mpar.n):
    solution.append(odeint(dif, a_0(i), mpar.t))
#print ("rfn", solution[1][mpar.L -1,3])


#Решаем сиситему 18 диф. уравнений с лазером
t2 = np.linspace(mpar.u, mpar.u + 325, 100000)

def a_00(i):
    a_00 = solution[i][mpar.L -1]
    return a_00
solution2 = []


# Решения 18 уравнений для n экспериментов
for i in range(mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))

end = time.time()
vt = end-start
print('Время выполнения:',vt)
print('Среднее время выполнения одного эксперимента:',vt/mpar.nn)

In [ ]:
first = time.time()
solution2 = []
t2 = linspace(mpar.u, mpar.u + 325, 100000)
def a_00(i):
    a_00 = solution[i][mpar.L -1]
    return a_00
for i in range (mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))
firstend = time.time()
print(firstend-first)

In [ ]:
first = time.time()
solution2 = []
t2 = linspace(mpar.u, mpar.u + 325, 100000)
def a_00(i):
    a_00 = np.array([solution[i][mpar.L - 1, 0], solution[i][mpar.L - 1, 1], solution[i][mpar.L - 1, 2], 
                     solution[i][mpar.L - 1, 3], solution[i][mpar.L - 1, 4], solution[i][mpar.L - 1, 5],
                     solution[i][mpar.L - 1, 6], solution[i][mpar.L - 1, 7], solution[i][mpar.L - 1, 8],
                     solution[i][mpar.L - 1, 9], solution[i][mpar.L - 1, 10], solution[i][mpar.L - 1, 11],
                     solution[i][mpar.L - 1, 12], solution[i][mpar.L - 1, 13], solution[i][mpar.L - 1, 14],
                     solution[i][mpar.L - 1, 15], solution[i][mpar.L - 1, 16], solution[i][mpar.L - 1, 17]])
    return a_00
for i in range (mpar.n):
    solution2.append(odeint(dif2, a_00(i), t2))
firstend = time.time()
print(firstend-first)